These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [165]:
import requests
import datetime

API_KEY = ''

def create_dict():
    """Create a dict with the key as the trade date 
       and the value as a dict with the trade information
    """
    afx_dict = {}
    column_list = json_data['dataset']['column_names']
    column_list.remove('Date')

    for item in json_data['dataset']['data']:
        values_list = item
        trade_date_string = values_list[0] 
        trade_date = date.fromisoformat(trade_date_string)
        values_list.remove(trade_date_string)
        trade_data = dict(list(zip(column_list, values_list)))
        afx_dict[trade_date] = trade_data
    
    return afx_dict

def data_float_list(trades_dict, column_name):
    """Return a list of data from the given column as float types.
       Convert nonetype to zero.
    
       Keyword arguments:
       trades_dict -- dataframe containing opening prices
       column_name -- column name to retrieve
    """
    return [float(trade_data[column_name] or 0) for trade_date, trade_data in afx_dict.items()]

def daily_changes(trades_dict):
    """Return a list of daily changes
    
       Keyword arguments:
       trades_dict -- dataframe containing opening prices
    """
    return [(trade_data['High'] - trade_data['Low']) for trade_date, trade_data in afx_dict.items()]

def max_diff(trades_dict, start_date, end_date):
    """Returns the largest difference between any two consecutive trading days.
       Calculates the difference rather than use the diff parameter.
       Maintains the sign, positive or negative, of the result.
    
       Keyword arguments:
       trades_dict -- dataframe containing opening prices
    """
    date1 = start_date
    date2 = start_date + datetime.timedelta(days=1)
    diff = 0
    max_diff = 0
    while date2 <= end_date:
        if afx_dict.get(date1) and afx_dict.get(date2):
            diff = afx_dict[date2]['Close'] - afx_dict[date1]['Close']
            if abs(diff) > abs(max_diff):
                max_diff = diff
        date1 = date2
        date2 = date2 + datetime.timedelta(days=1)

    return max_diff

def find_median(num_list):
    median = 0
    num_list.sort()
    if len(num_list)%2==0:
        middle_index = len(num_list)//2
        median = (num_list[middle_index -1] + num_list[middle_index])/2
    else:
        middle_index = len(num_list)//2 
        median = num_list[middle_index]
    return median

if __name__ == '__main__':
    
    #1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017     
    url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?'
    url = url + 'start_date=2017-01-01&end_date=2017-12-31&'
    url = url + 'api_key=' + API_KEY
    r = requests.get(url)
    json_data = r.json()

    data_start_date = date.fromisoformat('2017-01-02')
    data_end_date = date.fromisoformat('2017-12-31')

    #2. Convert the returned JSON object into a Python dictionary.
    afx_dict = create_dict()

    #3. Calculate what the highest and lowest opening prices were for the stock in this period.
    opening_prices = data_float_list(afx_dict, 'Open')
    print('Highest opening price: {:.2f}'.format(max(opening_prices)))
    print('Lowest opening price: {:.2f}'.format(min(opening_prices)))

    #4. What was the largest change in any one day (based on High and Low price)?
    print('Largest daily change: {:.2f}'.format(max(daily_changes(afx_dict))))

    #5. What was the largest change between any two days (based on Closing Price)?
    largest_change = max_diff(afx_dict, data_start_date, data_end_date)
    print('Largest absolute change between any two days: {:.2f}'.format(largest_change))

    #6. What was the average daily trading volume during this year?
    daily_vol = data_float_list(afx_dict, 'Traded Volume')
    print('Average daily trading volume: {:.2f}'.format(sum(daily_vol) / len(daily_vol)))

    #7. (Optional) What was the median trading volume during this year. 

    median = find_median([trade_data['Traded Volume'] for trade_date, trade_data in afx_dict.items()])
    print('Median trading volume: {:.2f}'.format(median))


Highest opening price: 53.11
Lowest opening price: 0.00
Largest daily change: 2.81
Largest absolute change between any two days: -2.56
Average daily trading volume: 89124.34
Median trading volume: 76286.00
